# Validate ALO Derivatives

*This notebook uses finite differences to validate the gradient and hessian of ALO.*

## Import Dependencies

In [1]:
import peak_engines
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_breast_cancer
import numpy as np

## Load a Data Set to Validate Against

In [2]:
X, y = load_breast_cancer(return_X_y=True)
X = StandardScaler().fit_transform(X)
delta = 1.0e-5

## Ridge Regularization
Test logistic regression with a ridge regularizer.

In [3]:
objective = peak_engines.LogisticRegressionModel().impl_.make_objective(X, y)
alpha = 2.5

### Validate Gradient

In [4]:
g = objective.compute_gradient([alpha])
g_approx = (objective.compute_value([alpha+delta]) - objective.compute_value([alpha])) / delta
print(g[0], g_approx)

9.023159877606496 9.019189140246908


### Validate Hessian

In [5]:
H = objective.compute_hessian([alpha])
H_approx = (objective.compute_gradient([alpha+delta]) - objective.compute_gradient([alpha])) / delta
print(H[0,0], H_approx[0])

0.24116954853244 0.24161082023965716


## Bridge Regularization
Test logistic regression with a bridge regularizer that requires multiple hyperparameters.

In [6]:
objective = peak_engines.LogisticRegressionModel(penalty='bridge').impl_.make_objective(X, y)
alpha = 2.5
beta = 1.25
params = np.array([2.5, 1.25])

### Validate Gradient

In [7]:
g = objective.compute_gradient(params)
g_approx = np.zeros(2)
f0 = objective.compute_value(params)
for j in range(2):
    params_prime = np.array(params)
    params_prime[j] += delta
    g_approx[j] = (objective.compute_value(params_prime) - f0) / delta
for j, (gj, g_approxj) in enumerate(zip(g, g_approx)):
    print(j, gj, g_approxj)

0 7.004190690732526 7.004206040761573
1 -10.46313945114182 -10.463306814756379


### Validate Hessian

In [8]:
H = objective.compute_hessian(params)
H_approx = np.zeros((2, 2))
g0 = objective.compute_gradient(params)
for j1 in range(2):
    for j2 in range(j1+1):
        params_prime = np.array(params)
        params_prime[j2] += delta
        g_prime = objective.compute_gradient(params_prime)
        H_approx[j1, j2] = (g_prime[j1] - g0[j1]) / delta
        H_approx[j2, j1] = H_approx[j1, j2]
for j1 in range(2):
    for j2 in range(2):
        print(j1, j2, H[j1, j2], H_approx[j1, j2])

0 0 0.2036221356205917 0.2036184313602973
0 1 -8.337466000152448 -8.337484601916856
1 0 -8.337466000152448 -8.337484601916856
1 1 5.746497467334356 5.746726380273514
